In [1]:
#First, import the libraries we will be using for scraping
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
#Open up a connection to wikipedia articla that lists all counties in PA and make BS object
html = urlopen('https://en.wikipedia.org/wiki/List_of_counties_in_Pennsylvania')
bysc = BeautifulSoup(html.read(), "lxml")

Store the county names in a list container.

In [3]:
county_list = []

In [4]:
table_a_tags = bysc.table.find_all('a')
for tag in table_a_tags:
    cut_point = tag.text.find('County')
    if cut_point > -1:
        county = str(tag.text[0:cut_point]).strip()
        #print(tag.text[0:cut_point]) DEBUG LINE
        county_list.append(county)

Now, we will scrape https://www.health.pa.gov/topics/disease/Pages/Coronavirus.aspx to get numbers on confirmed cases in PA by county.

Changed to this page as of 3/19: https://www.health.pa.gov/topics/disease/coronavirus/Pages/Cases.aspx

In [5]:
html_confirmed = urlopen('https://www.health.pa.gov/topics/disease/coronavirus/Pages/Cases.aspx')
bysc_confirmed = BeautifulSoup(html_confirmed.read(), "lxml")

In [6]:
#Create empty containers for storing column values
#Created as a set originally due to complications with Allegheny
counties_with_confirmed_cases = set()
confirmed_case_count = []
confirmed_death_count = []

In [7]:
#Needed to adjust to handle changes made to the PA website. There are now multiple tables on their COVID-19 page
all_tables = bysc_confirmed.find_all('table')

#Grab the table with the information we want
confirmed_table = all_tables[3]

#isolate the information on counties and confirmed cases within the table
table_tags = confirmed_table.find_all('td')

In [8]:
#Find the confirmed cases and deaths by county below
idx = 1 #For isolating every 3rd column as deaths

for tag in table_tags:
    #First 3 items are just headings, skip past them
    if idx <= 3:
        idx += 1
        continue
    item = tag.text.strip()
    #Strip off leading chars for some table items that cause issues
    if item.find("\u200b") != -1:
        item = item.replace("\u200b", "")
    if item.find("\xa0") != -1:
        item = item.replace("\xa0", "")
    #This is the deaths column. Recently added column that I'm now incorporating into spreadsheet (3/19/2020)
    if idx %3 == 0:
        #Currently, its null if 0 and has an integer if any deaths exist in county
        if item.isnumeric():
            confirmed_death_count.append(item)
        else:
            confirmed_death_count.append(0)
        idx += 1
    else:
        #Add evens to county set, odds to confirmed case count
        if item.isalpha():
            counties_with_confirmed_cases.add(item)
        if item.isnumeric():
            confirmed_case_count.append(item)
        idx += 1


In [9]:
#Convert the set to a list so that we can sort it below alphabetically
counties_with_confirmed_cases = [county for county in counties_with_confirmed_cases]
counties_with_confirmed_cases = sorted(counties_with_confirmed_cases)

In [10]:
#This is for confirmed cases of covid-19
confirmed_county_dictionary = {}
#Additional dict as of 3-19-20 for confirmed deaths
confirmed_death_dict = {}

counter = 0
for county in counties_with_confirmed_cases:
    confirmed_county_dictionary.update({county : confirmed_case_count[counter]})
    confirmed_death_dict.update({county : confirmed_death_count[counter]})
    counter += 1
    

In [11]:
#Debugging print line to ensure results are accurate
#print(confirmed_county_dictionary)
#print(confirmed_death_dict)

In [12]:
#Now, import numpy and pandas for joining all this information into dataframe
import numpy as np
import pandas as pd
import datetime

In [13]:
#Create an empty numpy array of zeros with length of total counties in PA
count_array = np.zeros(len(county_list))
confirmed_deaths_array = np.zeros(len(county_list))

#Update the corresponding position in arrays for counties that have reported cases on PA website
for idx, county in enumerate(county_list):
    if confirmed_county_dictionary.get(county) != None:
        count_array[idx] = confirmed_county_dictionary.get(county)
        confirmed_deaths_array[idx] = confirmed_death_dict.get(county)

In [14]:
#Create the pandas dataframe
df = pd.DataFrame({'County':county_list,
                   'Cumulative Confirmed Cases':count_array,
                   'Cumulative Confirmed Deaths' : confirmed_deaths_array 
})

df.insert(3, "As of", str(pd.Timestamp.now())[:10]) 


In [15]:
#Check the confirmed case county by county
#print(confirmed_county_dictionary) #Debugging line

In [16]:
#Check the contents of the dataframe
df

County  Cumulative Confirmed Cases  Cumulative Confirmed Deaths  \
0            Adams                         8.0                          0.0   
1        Allegheny                       158.0                          2.0   
2        Armstrong                         1.0                          0.0   
3           Beaver                        14.0                          0.0   
4          Bedford                         0.0                          0.0   
5            Berks                        65.0                          0.0   
6            Blair                         1.0                          0.0   
7         Bradford                         2.0                          0.0   
8            Bucks                       124.0                          0.0   
9           Butler                        26.0                          1.0   
10         Cambria                         1.0                          0.0   
11         Cameron                         0.0                          0.0   
12          Carbon                         2.0                          0.0   
13          Centre                        11.0                          0.0   
14         Chester                       107.0                          0.0   
15         Clarion                         0.0                          0.0   
16      Clearfield                         2.0                          0.0   
17         Clinton                         0.0                          0.0   
18        Columbia                         3.0                          0.0   
19        Crawford                         1.0                          0.0   
20      Cumberland                        16.0                          0.0   
21         Dauphin                        18.0                          0.0   
22        Delaware                       185.0                          3.0   
23             Elk                         0.0                          0.0   
24            Erie                         7.0                          0.0   
25         Fayette                         9.0                          0.0   
26          Forest                         0.0                          0.0   
27        Franklin                         5.0                          0.0   
28          Fulton                         0.0                          0.0   
29          Greene                         4.0                          0.0   
..             ...                         ...                          ...   
37         Lebanon                        12.0                          0.0   
38          Lehigh                        93.0                          2.0   
39         Luzerne                        55.0                          2.0   
40        Lycoming                         2.0                          0.0   
41          McKean                         0.0                          0.0   
42          Mercer                         4.0                          0.0   
43         Mifflin                         0.0                          0.0   
44          Monroe                        98.0                          2.0   
45      Montgomery                       374.0                          3.0   
46         Montour                         4.0                          0.0   
47     Northampton                        79.0                          3.0   
48  Northumberland                         1.0                          0.0   
49           Perry                         0.0                          0.0   
50    Philadelphia                       530.0                          2.0   
51            Pike                        23.0                          0.0   
52          Potter                         1.0                          0.0   
53      Schuylkill                        13.0                          0.0   
54          Snyder                         0.0                          0.0   
55        Somerset                         2.0                

In [17]:
#Export the dataframe to csv
#Grab the date from the dataframe 'As of' column
date = df['As of'][0]
df.to_csv('data/confirmed_cases_by_county_' + date + '.csv')